In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from Net_2 import *
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.preprocessing import StandardScaler

cat_cols=['Surname', 'Geography', 'Gender', 'Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited', 'binned_Age']
num_cols=['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

data = pd.read_csv('data_ex/train.csv')
# lab_CustomerId=LabelEncoder()
# lab_Surname=LabelEncoder()
lab_Gender=LabelEncoder()
lab_Geography=LabelEncoder()
lab_binned_Age=LabelEncoder()
# data['CustomerId']=lab_CustomerId.fit_transform(data['CustomerId'])
# data['Surname']=lab_Surname.fit_transform(data['Surname'])
data['Gender']=lab_Gender.fit_transform(data['Gender'])
data['Geography']=lab_Geography.fit_transform(data['Geography'])
data['binned_Age']=pd.cut(data['Age'], bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=[0, 1, 2, 3, 4, 5, 6, 7, 8])
data['binned_Age']=lab_binned_Age.fit_transform(data['binned_Age'])
scalar=StandardScaler()
data[num_cols]=scalar.fit_transform(data[num_cols])

![jupyter](./image/CustomerId.png)
![jupyter](./image/Surname.png)

In [13]:
#处理特征CustomerId与Surname
#按照上述图片中的界限分桶counts = 20,40,80
# 计算A特征的totalCounts
# 计算CustomerId的totalCounts
C_Id_counts = data['CustomerId'].value_counts()
data['CustomerId'] = data.groupby('CustomerId')['CustomerId'].transform('count')
# 根据totalCounts将CustomerId划分为四个区间
bins = [0, 20, 40, 80,C_Id_counts.max()+1]
labels = [0, 1, 2, 3]
data['CustomerId_binned'] = pd.cut(data['CustomerId'], bins=bins, labels=labels)

Surname_counts = data['Surname'].value_counts()
data['Surname'] = data.groupby('Surname')['Surname'].transform('count')
# 根据totalCounts将Surname划分为四个区间
bins = [0, 200, 800,Surname_counts.max()+1]
labels = [0, 1, 2]
data['Surname_binned'] = pd.cut(data['Surname'], bins=bins, labels=labels)

In [5]:
X=data.drop(['Exited', 'id' , 'CustomerId' , 'Surname'], axis=1).values
y=data['Exited'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

In [22]:
net = Network([X_train.shape[1],13,1],[sigmoid,sigmoid])  # 修改网络结构以匹配Sequential模型
history = net.train(X_train, y_train, 10, 32, 0.1)

In [23]:
# 评估网络性能
acc, precision, recall, f1,cost= net.evaluate(zip(X_test, y_test))
print('acc:', acc)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)
print('cost:', cost)

In [24]:
# 将history转置，使得每个指标为一个列表
acc, precision, recall, f1, _ = zip(*history)

epochs = range(1, len(history) + 1)

# 绘制每个指标的曲线
plt.figure(figsize=(12, 6))

plt.plot(epochs, acc, 'b', label='Accuracy')
plt.plot(epochs, precision, 'g', label='Precision')
plt.plot(epochs, recall, 'r', label='Recall')
plt.plot(epochs, f1, 'c', label='F1 Score')
#x轴刻度为整数
plt.xticks(epochs)
plt.title('Training Metrics Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()

plt.grid(True)
plt.savefig("image/final.svg")

In [25]:
#预测test数据
lab = LabelEncoder()
data_test = pd.read_csv('data_ex/test.csv')
data_test['Gender']=lab_Gender.transform(data_test['Gender'])
data_test['Geography']=lab_Geography.transform(data_test['Geography'])
data_test['binned_Age']=pd.cut(data_test['Age'], bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=[0, 1, 2, 3, 4, 5, 6, 7, 8])
data_test['binned_Age']=lab_binned_Age.transform(data_test['binned_Age'])
data_test[num_cols]=scalar.transform(data_test[num_cols])
data_test['CustomerId'] = data_test.groupby('CustomerId')['CustomerId'].transform('count')
bins = [0, 20, 40, 80,C_Id_counts.max()+1]
labels = [0, 1, 2, 3]
data_test['CustomerId_binned'] = pd.cut(data_test['CustomerId'], bins=bins, labels=labels)
data_test['Surname'] = data_test.groupby('Surname')['Surname'].transform('count')
bins = [0, 200, 800,Surname_counts.max()+1]
labels = [0, 1, 2]
data_test['Surname_binned'] = pd.cut(data_test['Surname'], bins=bins, labels=labels)
X= data_test.drop([ 'id', 'CustomerId', 'Surname'], axis=1).values
X

In [26]:
y_pred = np.array([net.Forward_prop(x) for x in X])
result = pd.DataFrame({'id':data_test['id'], 'Exited':y_pred[:,0]})
result.to_csv('result_final.csv',index=False)


In [27]:
#保存模型
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(net, f)